****Extract Glossary From Text****

**Part 1** - Extract Keywords and Definitions from Block of Text Using Gemini.

Includes langchain to address the Gemini API Key Problem

This is based on the following examples

 - https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/prompts/examples/text_extraction.ipynb
 - https://medium.com/@joansantoso/gemini-reshaping-the-nlp-task-for-extracting-knowledge-in-text-c0d5fdd4edd8
 - https://codelabs.developers.google.com/codelabs/genai-text-gen-java-palm-langchain4j#4

  

In [1]:
pip install -U langchain langchain-google-vertexai 

INFO: pip is looking at multiple versions of google-api-core[grpc] to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 27.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.8/355.8 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 1.34.1
    Uninstalling google-api-core-1.34.1:
      Successfully uninstalled google-api-core-1.34.1
  You can safely remove it manually.
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.14.0
    Uninstalling goo

**Check Version Nos of what was installed**

In [2]:
!pip show langchain langchain-core langchain-google-vertexai neo4j google-cloud-aiplatform google-generativeai google-ai-generativelanguage

Name: langchain
Version: 0.2.7
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /opt/conda/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
---
Name: langchain-core
Version: 0.2.12
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /opt/conda/lib/python3.10/site-packages
Requires: jsonpatch, langsmith, packaging, pydantic, PyYAML, tenacity
Required-by: langchain, langchain-google-vertexai, langchain-text-splitters
---
Name: langchain-google-vertexai
Version: 1.0.6
Summary: An integration package connecting Google VertexAI and LangChain
Home-page: https://github.com/langchain-ai/langchain-google
Author: 
Author-email: 
License: MI

**Check Jupyter Version No**

In [3]:
!jupyter --version

Selected Jupyter core packages...
IPython          : 8.21.0
ipykernel        : 6.29.4
ipywidgets       : 8.1.3
jupyter_client   : 7.4.9
jupyter_core     : 5.7.2
jupyter_server   : 1.24.0
jupyterlab       : 3.4.8
nbclient         : 0.10.0
nbconvert        : 7.16.4
nbformat         : 5.10.4
notebook         : 6.5.7
qtconsole        : not installed
traitlets        : 5.14.3


**Check Python Version/Path** - *Expect 3.10.14*

In [4]:
import sys
import platform
print(sys.version)
print(platform.python_version())
print(sys.path)

3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:45:18) [GCC 12.3.0]
3.10.14
['/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/opt/conda/lib/python3.10/site-packages']


**Now for the Imports**

This time we are isloating Vertex AI

In [5]:
import os
import requests

#from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_google_vertexai import ChatVertexAI
from langchain_core.documents import Document

**Diagnostic Methods**

In [6]:
## Vertex AI Model Diagnostic Dump
def print_llm(llm):
    print(f"llm: {llm}")
    print(f"name: {llm.model_name}")
    print(f"examples: {llm.examples}") 
    print(f"tuned_model_name: {llm.tuned_model_name}") 
    print(f"convert_system_message_to_human: {llm.convert_system_message_to_human}") 
    print(f"max_output_tokens: {llm.max_output_tokens}") 
    print(f"top_p: {llm.top_p}") 
    print(f"top_k: {llm.top_k}") 
    print(f"credentials: {llm.credentials}")     
    print(f"n: {llm.n}") 
    print(f"streaming: {llm.streaming}") 
    print(f"safety_settings: {llm.safety_settings}")     
    print(f"api_transport: {llm.api_transport}") 
    print(f"api_endpoint: {llm.api_endpoint}") 
 
    print('properties') 
    print('----------') 
    
    print(f"_llm_type: {llm._llm_type}") 
    #print(f"is_codey_model: {llm.is_codey_model}")     
    print(f"_is_gemini_model: {llm._is_gemini_model}")
    print(f"_identifying_params: {llm._identifying_params}")     
    print(f"_default_params: {llm._default_params}") 
    print(f"_user_agent: {llm._user_agent}") 



**Connect to Google LLMs**

*Least Privilege Security.*

The Notebook is "owned" by a bespoke Service Account created in terrafrom for this purpose.

Minimal permisisons are added (also via terraform) via predefined roles (esp. Vertex) as required.

This is typically triggered by a PERMISSION DENIED error

In [7]:
import os
# Set It - will require regeneration
os.environ['GOOGLE_API_KEY'] = '32f9af541e5febaadd9eeb230be8cef11dc9d171'
# Access the environment variable later in your code
api_key = os.environ['GOOGLE_API_KEY']
print(api_key)

32f9af541e5febaadd9eeb230be8cef11dc9d171


****Enable Langchain Debugging****

See: https://python.langchain.com/v0.1/docs/guides/development/debugging/

In [8]:
from langchain.globals import set_debug   

set_debug(False)

**Create The LLMs**

Both *Gemini* & *Chat Bison* were created.

Chat Bison malfunctioned so has been abandoned FTTB 

Sourced from here: https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models 

In [9]:
# Avalable Model Variables

# Gemini 1.5 Pro (Preview)
# 404 Publisher Model `projects/nlp-dev-6aae/locations/us-central1/publishers/google/models/gemini-1.5-pro` not found.
gemini_1pt5_proOnVertex = ChatVertexAI(model="gemini-1.5-pro-001")



# Gemini 1.0 Pro
# This works with Errors - chunking
gemini_1pt0_proOnVertex = ChatVertexAI(model="gemini-1.0-pro")

gemini_proOnVertex = ChatVertexAI(model="gemini-pro")


**Choose which model we are using**

In [10]:
chat_llm = gemini_1pt5_proOnVertex

*Check Model Properties*

In [11]:
print_llm(chat_llm)

llm: project='nlp-dev-6aae' model_name='gemini-1.5-pro-001' model_family=<GoogleModelFamily.GEMINI_ADVANCED: '2'> full_model_name='projects/nlp-dev-6aae/locations/us-central1/publishers/google/models/gemini-1.5-pro-001' client_options=ClientOptions: {'api_endpoint': 'us-central1-aiplatform.googleapis.com', 'client_cert_source': None, 'client_encrypted_cert_source': None, 'quota_project_id': None, 'credentials_file': None, 'scopes': None, 'api_key': None, 'api_audience': None} default_metadata=()
name: gemini-1.5-pro-001
examples: None
tuned_model_name: None
convert_system_message_to_human: False
max_output_tokens: None
top_p: None
top_k: None
credentials: None
n: 1
streaming: False
safety_settings: None
api_transport: None
api_endpoint: None
properties
----------
_llm_type: vertexai
_is_gemini_model: True
_identifying_params: {'model_name': 'gemini-1.5-pro-001', 'temperature': 0.0, 'max_output_tokens': 128, 'candidate_count': 1, 'top_k': 40, 'top_p': 0.95}
_default_params: {'temperatur

****Test Data****

This is an extract from the Transcript of a GCP ML Engineer Professional Certification Training Course

In [12]:
from langchain_core.documents import Document

tutorial_text = """
Transcript
Hello, everyone.
So this video will be will be a gentle and brief introduction on what is data science, what is artificial
intelligence, machine learning and deep learning.
And this will be a short video just in like getting ourselves familiarized with the basic definition
of all these four concepts.
So let's get started.
So just remember this diagram and I will be covering the definition also.
So data science is something which which overlaps with all deep learning, machine learning and artificial
intelligence.
And this is a very, very wide and extensive field.
It contains a lot of like mathematics, maths statistics and other things.
But this, these three things, so deep learning is a very, very, very specialized field.
And deep learning, we can say, is a subset of machine learning.
And machine learning is all about computers learning or machines learning about the different like they
are learning about the system and coming up with a way to predict or predict the outcome.
So deep learning is very specialized.
It's a subset.
It's a it's a specialized field of machine learning.
And same way machine learning is a superset of deep learning.
But machine learning is a subset of artificial intelligence, which is a very huge and wide field.
So deep learning is part of machine learning.
Machine learning is a part of artificial intelligence.
And we can say deep learning is also a very small subset of artificial intelligence.
So just think of just remember this diagram and you will be like able to distinguish between these three
concepts and data.
Science is something where separate, but it has a connection with all three topics.
So data science.
So data science is a field that involves using statistical and computational techniques to extract insights
and knowledge from data.
It is a multidisciplinary disciplinary field that combines elements from computer science statistics
and domain expertise in order to understand and analyze complex data sets.
Data.
Scientists use a variety of tools and techniques to analyze data, including machine learning, algorithms,
statistics, statistical analysis and visualization tools.
So Data Science Part two.
So they work with structured and unstructured data and use a variety of methods to clean prepare and
process the data for analysis.
The goal of data science is to extract useful insights and knowledge from data that can be used to inform
decision making and solve problems.
Data scientists may work in a variety of industries, including finance, healthcare and e-commerce,
and may use their skills to tackle a wide range of problems, such as predicting customer behaviour,
optimizing supply chain logistics or improve health healthcare outcomes.
So next is machine learning.
Machine learning is a method of teaching computers to learn from data without being explicitly programmed.
It is a subset of artificial intelligence that focuses on the development of computer programs that
can access data and use it to learn for themselves.
There are three main types of machine learning.
Supervised Learning.
Unsupervised learning and reinforcement learning.
In supervised learning.
The computer is given a set of labeled training data and a set of desired outputs, and the algorithm
learns to map the inputs to the outputs.
For example, a supervised learning algorithm might be trained to classify email as spam or not spam
based on a set of labeled examples.
In unsupervised learning.
The algorithm is given a set of data without any labels, and it must find patterns and relationships
in the data.
One example of unsupervised learning is clustering, where the algorithm groups similar data points
together.
In reinforcement learning, the algorithm learns by interacting with its environment and receiving towards
rewards or penalties for certain actions.
So reinforcement enforcement learning is mostly used in like automatic, like self-driving cars and
robots and race cars where there is no human involved.
So or in some kind of game where you want to program like some, uh, some toy or something and it will
keep like going through multiple, multiple races and see, every time it, it has an accident or it
hits, it learns and it creates a reinforcement, like a reward.
What is the reward and what is a penalty?
And it will just keep doing that and it will learn from that environment.
So this is the type of learning used by, as I mentioned, self-driving cars and robots to learn how
to navigate their environments.
So next is artificial intelligence.
Artificial intelligence refers to the ability of computers and machines to perform tasks that would
normally require human intelligence, such as learning problem solving and decision making.
So as I mentioned, this is a higher form of of machine learning or deep learning, but this is a higher
form.
So this is a machine learning involved, plus some some kind of intelligence like some kind of problem
solving or decision making skills.
So there are several types of artificial intelligence, including narrow or weak artificial intelligence.
This type of AI is used to perform a specific task, such as playing a game or recognizing speech.
It is not generally capable of performing other tasks, general or strong AI.
This type of AI is capable of performing a wide range of tasks and can exhibit human like intelligence.
And last is artificial superintelligence.
This is a hypothetical future form of AI that would surpass human intelligence in a wide range of domains.
So next, again, continuing artificial intelligence is achieved through a combination of machine learning
algorithms and other techniques that allow computers to process and analyze data in order to make decisions
and take actions.
Some common applications of AI include image and speech recognition, natural language processing and
autonomous vehicles.
He has the potential to revolutionize.
Revolutionize many industries and has already had a significant impact on fields such as healthcare,
finance and manufacturing.
Next last concept is deep learning.
Deep learning is a subfield of machine learning that is inspired by the structure and function of the
brain, especially the neural networks that make up the brain.
It involves training artificial neural networks on a large dataset, allowing allowing the network to
learn and make intelligent decisions on its own.
Deep learning algorithms are constructed using multiple layers of artificial neural networks with each
layer processing the input data and passing it on to the next layer.
So part to this hierarchical structure allows the algorithm to learn and represent very complex patterns
in the data.
Deep learning has been successful in a wide range of applications, including image and speech recognition,
natural language processing and even playing games like Chess and Go.
It has also been used to improve the performance of machine learning algorithms in other fields such
as healthcare and finance.
So just I'll try to explain in the simplest possible way.
In manual programming.
So in like in regular programming, you tell the computer like what is the algorithm equation formula
or rules, for example, I'll just take one example.
For example, you tell like you give some input and you say one plus two equal to three.
So you just program, you just write a program where that it gets two inputs and outcome or output wil
be three.
So you just you just tell that this is the formula, this is the equation, this is the rule.
So the computer will just perform this.
There is no learning involved.
Just you tell this is the the formula, right?
But in machine learning machines learn the algorithm equation or the formula by themselves, by different
permutations.
So for example.
You just feed in the data.
You just say input that to input, for example, A and B and the outcome or output.
Let's just call let's call it C, So you just say a B is the input and output will be six and you just
feed in more data like three and four and output is is 12 and same.
You have given the three set of data, just very simplistic example like even 1 or 2 will do.
But just to, just to put my point across.
So you just feed in, for example, the third set of data like four into four and five A and B and the
output will be 20 in this case.
So you tell the computer that, tell me what is the formula like?
What is the equation?
And in this case, for simplistic, it can be plus minus, multiply or divide.
So so machines will go through all the data and will figure out, okay, this is like a multiplication
multiplication.
So this is very simplistic way, but in in real world will be like a very complex problem.
It can have exponential logarithms, root square, root, quadratic, root, all kind of things can
go.
It can be there will be multiple like features where multiple inputs it can span ten, 2000s also and
there will be one single output.
And the mathematical operator can be not for it can be any number of it can be root, square, root
log exponential.
So the computer will just will go through all the data and finally come up with the most optimal like
generic equation.
And then that's how they learn.
So we don't have to program and figure out all the logic and all.
So for complex real world problems, you feed in lots of data and expected outcomes and machines will
come up with the algorithm equation or the formula.
So you can just some some people will call it algorithm rules equation problem.
So just think of anything that can represent some real world problem and what could be the expected
solution.
So that's all.
Thanks for watching.
"""

documents = [Document(page_content=tutorial_text)]

https://python.langchain.com/v0.2/docs/integrations/llms/google_ai/

In [16]:
prompt_text='''
1.You are a Named Entity Recognition in English Language.
2.Do some analysis to extract the Keywords, that is important entities, from the text. 
3.Assign a Category for each entity, in Upper Case. Assign MISC for Entites which cannot be classified easily.
4.Extract a Summary Definition for each Keyword Extracted. The Summary Definition must not be more than 100 words length. If no definition is available, assign NOT AVAILABLE. 
4.Return this result as JSON for each Keyword. Thus each Keyword will have 3 JSON Attributes: Entity, Category and Definition.
Analyze the following sentences: "'
'''

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# split documents into text and embeddings

text_splitter = RecursiveCharacterTextSplitter(
   chunk_size=512, 
   chunk_overlap=20,
   length_function=len,
   is_separator_regex=False
)

chunks = text_splitter.split_documents(documents)

print(f"No. chunks{len(chunks)}")

No. chunks21


In [18]:
response =  chat_llm.invoke(prompt_text+tutorial_text+'"')
print(response)

content='```json\n[\n  {\n    "Entity": "Data Science",\n    "Category": "FIELD",\n    "Definition": "Data science uses statistical and computational techniques to extract insights from data. It combines computer science, statistics, and domain expertise to analyze complex datasets. Data scientists utilize tools like machine learning algorithms, statistical analysis, and visualization tools to work with structured and unstructured data."\n  },\n  {\n    "Entity": "Machine Learning",\n    "Category": "FIELD",\n    "Definition": "Machine learning teaches computers to learn from data without explicit programming. It\'s a subset of' response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGOR

In [19]:
graph_extract_prompt = '''
# Knowledge Graph Instructions for Gemini
## 1. Overview
You are a top-tier algorithm designed for extracting information in structured 
formats to build a knowledge graph.
Try to capture as much information from the text as possible without 
sacrificing accuracy. Do not add any information that is not explicitly 
mentioned in the text
- **Nodes** represent entities and concepts.
- The aim is to achieve simplicity and clarity in the knowledge graph, making it
accessible for a vast audience.
## 2. Labeling Nodes
- **Consistency**: Ensure you use available types for node labels.
Ensure you use basic or elementary types for node labels.
- For example, when you identify an entity representing a person, 
always label it as **'person'**. Avoid using more specific terms 
like 'mathematician' or 'scientist'
  - **Node IDs**: Never utilize integers as node IDs. Node IDs should be 
names or human-readable identifiers found in the text.
- **Relationships** represent connections between entities or concepts.
Ensure consistency and generality in relationship types when constructing 
knowledge graphs. Instead of using specific and momentary types 
such as 'BECAME_PROFESSOR', use more general and timeless relationship types 
like 'PROFESSOR'. Make sure to use general and timeless relationship types!
## 3. Coreference Resolution
- **Maintain Entity Consistency**: When extracting entities, it's vital to 
ensure consistency.
    If an entity, such as John Doe, is mentioned multiple times in the text 
but is referred to by different names or pronouns (e.g., Joe, he),
always use the most complete identifier for that entity throughout the 
knowledge graph. In this example, use John Doe as the entity ID.
Remember, the knowledge graph should be coherent and easily understandable, 
so maintaining consistency in entity references is crucial.
## 4. Strict Compliance
Adhere to the rules strictly. Non-compliance will result in termination.
Analyze the following text corpus: "'
'''

In [20]:
graph_extract_corpus = """
Current Professional Machine Learning Engineer Certification exam guide
A Professional Machine Learning Engineer builds, evaluates, productionizes, and optimizes ML models by using Google Cloud technologies and knowledge of proven models and techniques. The ML Engineer handles large, complex datasets and creates repeatable, reusable code. The ML Engineer considers responsible AI and fairness throughout the ML model development process, and collaborates closely with other job roles to ensure long-term success of ML-based applications. The ML Engineer has strong programming skills and experience with data platforms and distributed data processing tools. The ML Engineer is proficient in the areas of model architecture, data and ML pipeline creation, and metrics interpretation. The ML Engineer is familiar with foundational concepts of MLOps, application development, infrastructure management, data engineering, and data governance. The ML Engineer makes ML accessible and enables teams across the organization. By training, retraining, deploying, scheduling, monitoring, and improving models, the ML Engineer designs and creates scalable, performant solutions.
Note: The exam does not directly assess coding skill. If you have a minimum proficiency in Python and Cloud SQL, you should be able to interpret any questions with code snippets.
Register now
The Professional Machine Learning Engineer exam does not cover generative AI, as the tools used to develop generative AI-based solutions are evolving quickly. If you are interested in generative AI, please refer to the Introduction to Generative AI Learning Path (all audiences) or the Generative AI for Developers Learning Path (technical audience). If you are a partner, please refer to the Gen AI partner courses: Introduction to Generative AI Learning Path, Generative AI for ML Engineers, and Generative AI for Developers.
Section 1: Architecting low-code ML solutions (~12% of the exam)
1.1 Developing ML models by using BigQuery ML. Considerations include:
Building the appropriate BigQuery ML model (e.g., linear and binary classification, regression, time-series, matrix factorization, boosted trees, autoencoders) based on the business problem
Feature engineering or selection by using BigQuery ML
Generating predictions by using BigQuery ML
1.2 Building AI solutions by using ML APIs. Considerations include:
Building applications by using ML APIs (e.g., Cloud Vision API, Natural Language API, Cloud Speech API, Translation)
Building applications by using industry-specific APIs (e.g., Document AI API, Retail API)
1.3 Training models by using AutoML. Considerations include:
Preparing data for AutoML (e.g., feature selection, data labeling, Tabular Workflows on AutoML)
Using available data (e.g., tabular, text, speech, images, videos) to train custom models
Using AutoML for tabular data
Creating forecasting models using AutoML
Configuring and debugging trained models
Section 2: Collaborating within and across teams to manage data and models (~16% of the exam)
2.1 Exploring and preprocessing organization-wide data (e.g., Cloud Storage, BigQuery, Spanner, Cloud SQL, Apache Spark, Apache Hadoop). Considerations include:
Organizing different types of data (e.g., tabular, text, speech, images, videos) for efficient training
Managing datasets in Vertex AI
Data preprocessing (e.g., Dataflow, TensorFlow Extended [TFX], BigQuery)
Creating and consolidating features in Vertex AI Feature Store
Privacy implications of data usage and/or collection (e.g., handling sensitive data such as personally identifiable information [PII] and protected health information [PHI])
2.2 Model prototyping using Jupyter notebooks. Considerations include:
Choosing the appropriate Jupyter backend on Google Cloud (e.g., Vertex AI Workbench, notebooks on Dataproc)
Applying security best practices in Vertex AI Workbench
Using Spark kernels
Integration with code source repositories
Developing models in Vertex AI Workbench by using common frameworks (e.g., TensorFlow, PyTorch, sklearn, Spark, JAX)
2.3 Tracking and running ML experiments. Considerations include:
Choosing the appropriate Google Cloud environment for development and experimentation (e.g., Vertex AI Experiments, Kubeflow Pipelines, Vertex AI TensorBoard with TensorFlow and PyTorch) given the framework
Section 3: Scaling prototypes into ML models (~18% of the exam)
3.1 Building models. Considerations include:
Choosing ML framework and model architecture
Modeling techniques given interpretability requirements
3.2 Training models. Considerations include:
Organizing training data (e.g., tabular, text, speech, images, videos) on Google Cloud (e.g., Cloud Storage, BigQuery)
Ingestion of various file types (e.g., CSV, JSON, images, Hadoop, databases) into training
Training using different SDKs (e.g., Vertex AI custom training, Kubeflow on Google Kubernetes Engine, AutoML, tabular workflows)
Using distributed training to organize reliable pipelines
Hyperparameter tuning
Troubleshooting ML model training failures
3.3 Choosing appropriate hardware for training. Considerations include:
Evaluation of compute and accelerator options (e.g., CPU, GPU, TPU, edge devices)
Distributed training with TPUs and GPUs (e.g., Reduction Server on Vertex AI, Horovod)
Section 4: Serving and scaling models (~19% of the exam)
4.1 Serving models. Considerations include:
Batch and online inference (e.g., Vertex AI, Dataflow, BigQuery ML, Dataproc)
Using different frameworks (e.g., PyTorch, XGBoost) to serve models
Organizing a model registry
A/B testing different versions of a model
4.2 Scaling online model serving. Considerations include:
Vertex AI Feature Store
Vertex AI public and private endpoints
Choosing appropriate hardware (e.g., CPU, GPU, TPU, edge)
Scaling the serving backend based on the throughput (e.g., Vertex AI Prediction, containerized serving)
Tuning ML models for training and serving in production (e.g., simplification techniques, optimizing the ML solution for increased performance, latency, memory, throughput)
Section 5: Automating and orchestrating ML pipelines (~21% of the exam)
5.1 Developing end-to-end ML pipelines. Considerations include:
Data and model validation
Ensuring consistent data pre-processing between training and serving
Hosting third-party pipelines on Google Cloud (e.g., MLFlow)
Identifying components, parameters, triggers, and compute needs (e.g., Cloud Build, Cloud Run)
Orchestration framework (e.g., Kubeflow Pipelines, Vertex AI Pipelines, Cloud Composer)
Hybrid or multicloud strategies
System design with TFX components or Kubeflow DSL (e.g., Dataflow)
5.2 Automating model retraining. Considerations include:
Determining an appropriate retraining policy
Continuous integration and continuous delivery (CI/CD) model deployment (e.g., Cloud Build, Jenkins)
5.3 Tracking and auditing metadata. Considerations include: 
Tracking and comparing model artifacts and versions (e.g., Vertex AI Experiments, Vertex ML Metadata)
Hooking into model and dataset versioning
Model and data lineage
Section 6: Monitoring ML solutions (~14% of the exam)
6.1 Identifying risks to ML solutions. Considerations include:
Building secure ML systems (e.g., protecting against unintentional exploitation of data or models, hacking)
Aligning with Google’s Responsible AI practices (e.g., biases)
Assessing ML solution readiness (e.g., data bias, fairness)
Model explainability on Vertex AI (e.g., Vertex AI Prediction)
6.2 Monitoring, testing, and troubleshooting ML solutions. Considerations include:
Establishing continuous evaluation metrics (e.g., Vertex AI Model Monitoring, Explainable AI)
Monitoring for training-serving skew
Monitoring for feature attribution drift
Monitoring model performance against baselines, simpler models, and across the time dimension
Common training and serving errors
"""

In [21]:
response =  chat_llm.invoke(graph_extract_prompt+graph_extract_corpus+'"')
print(response)

content='```json\n{\n  "nodes": [\n    {\n      "id": "Professional Machine Learning Engineer",\n      "type": "role"\n    },\n    {\n      "id": "ML models",\n      "type": "concept"\n    },\n    {\n      "id": "Google Cloud technologies",\n      "type": "technology"\n    },\n    {\n      "id": "proven models",\n      "type": "concept"\n    },\n    {\n      "id": "techniques",\n      "type": "concept"\n    },\n' response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity'